In [1]:
import openai
import pandas as pd

In [ ]:
client = openai.Client()

In [3]:
file = client.files.create(
    file=open("../sales_data.csv", "rb"),
    purpose="assistants"
)

In [4]:
print(f"File ID: {file.id}")

File ID: file-41g9YZh54gQ6xG3VLUqrE5


In [6]:
assistant = client.beta.assistants.create(
    name = "Analista de Dados",
    instructions = "Você é um analista que analisa dados sobre vendas",
    tools= [{"type": "code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids": [file.id]}},
    model = "gpt-4o"
)

In [33]:
# pergunta = "Qual o rating médio das vendas do supermercado?"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto."

In [34]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta
)

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/3685932091.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/3685932091.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [35]:
# Executa a Trhead
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions= "Nome e usuário premium"
)

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/1703593232.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [36]:
# Aguarda a Thread rodar
import time

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/3925348386.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [37]:
# Verificar a resposta

if run.status == "completed":
    mensagem = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagem)
else:
    print(f"Erro: {run.status}")    

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/1142915008.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagem = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_Q6vI57KYMGi4mlrABDlZZRz5', assistant_id='asst_nSVitkaGxMGWOcCzANdfb7D1', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-AoThkhV2q2MdUWtF3CGtze', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza mostrando o percentual de vendas por linha de produto. Cada fatia representa a proporção das vendas totais atribuídas a cada linha de produto. Se você tiver mais alguma dúvida ou precisar de outra análise, estou à disposição para ajudar!'), type='text')], created_at=1752562838, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_oibD6ItQJjis44OE75iAB8Sn', status=None, thread_id='thread_Bm0VvBqQB8HFbuOIEPUM0jiY'), Message(id='msg_INM2eL70LBP59NBhtzuka3pv', assistant_id='asst_nSVitkaGxMGWOcCzANdfb7D1', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(anno

In [39]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/1597243276.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [45]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")

    if step.step_details.type == "toool_calls":
        for tool_call in step.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem salva como files/{file.id}.png")



====Step message_creation


/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_85546/3493538446.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Para gerar um gráfico de pizza com o percentual de vendas por linha de produto, primeiro preciso inspecionar o conteúdo do arquivo enviado. Vou começar verificando o formato do arquivo e os dados contidos nele.

====Step tool_calls

====Step message_creation
O arquivo contém um conjunto de dados de vendas, incluindo informações como ID da fatura, cidade, tipo de cliente, gênero, linha de produto, preço unitário, quantidade, impostos, total e outros detalhes. Para gerar o gráfico de pizza, concentraremos nos dados das colunas "Product Line" e "Total" para calcular o percentual de vendas por linha de produto.

Vamos agora calcular os percentuais e criar o gráfico.

====Step tool_calls

====Step message_creation
Imagem salva como files/file-41g9YZh54gQ6xG3VLUqrE5.png
